In [ ]:
import pandas as pd
import mysql.connector

# Load CSV data
file_path = r'D:\dbms_project\bbb_data.csv'
data = pd.read_csv(file_path)

# Connect to MySQL database (replace with your own credentials)
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Murari@24",
    database="project"
)
cursor = conn.cursor()

# Create Tables
cursor.execute('''
CREATE TABLE IF NOT EXISTS Players (
    player_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    role VARCHAR(50),
    batting_style VARCHAR(50),
    bowling_style VARCHAR(50),
    date_of_birth DATE,
    country VARCHAR(50),
    career_matches INT,
    career_runs INT,
    career_wickets INT,
    batting_average FLOAT,
    bowling_economy FLOAT,
    strike_rate FLOAT,
    highest_score INT,
    best_bowling VARCHAR(10),
    debut_date DATE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Teams (
    team_id INT PRIMARY KEY AUTO_INCREMENT,
    team_name VARCHAR(100) UNIQUE NOT NULL
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Matches (
    match_id INT PRIMARY KEY,
    date DATE NOT NULL,
    venue VARCHAR(100),
    team1_id INT,
    team2_id INT,
    result VARCHAR(20),
    FOREIGN KEY (team1_id) REFERENCES Teams(team_id),
    FOREIGN KEY (team2_id) REFERENCES Teams(team_id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Scorecards (
    scorecard_id INT PRIMARY KEY AUTO_INCREMENT,
    match_id INT,
    team_id INT,
    runs INT,
    wickets INT,
    overs FLOAT,
    FOREIGN KEY (match_id) REFERENCES Matches(match_id),
    FOREIGN KEY (team_id) REFERENCES Teams(team_id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Statistics (
    stat_id INT PRIMARY KEY AUTO_INCREMENT,
    match_id INT,
    player_id INT,
    team_id INT,
    runs_scored INT,
    balls_faced INT,
    wickets_taken INT,
    overs_bowled FLOAT,
    FOREIGN KEY (match_id) REFERENCES Matches(match_id),
    FOREIGN KEY (player_id) REFERENCES Players(player_id),
    FOREIGN KEY (team_id) REFERENCES Teams(team_id)
)
''')

# Helper function to escape single quotes
def escape_quotes(value):
    return value.replace("'", "''") if isinstance(value, str) else value

# Insert unique teams into Teams table
teams = pd.concat([data['batting_team'], data['bowling_team']]).unique()
for team in teams:
    team_escaped = escape_quotes(team)
    cursor.execute(f"INSERT IGNORE INTO Teams (team_name) VALUES ('{team_escaped}')")

# Populate the Matches table
match_data = data[['match_id', 'season', 'start_date', 'venue', 'batting_team', 'bowling_team']].drop_duplicates()
for _, row in match_data.iterrows():
    cursor.execute(f'''
        INSERT IGNORE INTO Matches (match_id, date, venue, team1_id, team2_id)
        VALUES (
            {row['match_id']}, 
            '{escape_quotes(row['start_date'])}', 
            '{escape_quotes(row['venue'])}', 
            (SELECT team_id FROM Teams WHERE team_name = '{escape_quotes(row['batting_team'])}'), 
            (SELECT team_id FROM Teams WHERE team_name = '{escape_quotes(row['bowling_team'])}')
        )
    ''')

# Insert players into Players table
players = pd.concat([data['striker'], data['non_striker'], data['bowler']]).unique()
for player in players:
    player_escaped = escape_quotes(player)
    cursor.execute(f"INSERT IGNORE INTO Players (name) VALUES ('{player_escaped}')")

# Populate the Statistics table
for _, row in data.iterrows():
    # Find player and team IDs
    player_id_query = f"SELECT player_id FROM Players WHERE name = '{escape_quotes(row['striker'])}'"
    team_id_query = f"SELECT team_id FROM Teams WHERE team_name = '{escape_quotes(row['batting_team'])}'"
    
    cursor.execute(player_id_query)
    player_id = cursor.fetchone()[0]
    
    cursor.execute(team_id_query)
    team_id = cursor.fetchone()[0]
    
    # Insert statistics
    cursor.execute(f'''
        INSERT INTO Statistics (match_id, player_id, team_id, runs_scored, balls_faced, wickets_taken, overs_bowled)
        VALUES (
            {row['match_id']},
            {player_id},
            {team_id},
            {row['runs_off_bat']},
            1,  -- Assuming each entry is for a single ball faced
            {1 if pd.notna(row['wicket_type']) else 0},
            {1/6}  -- Assuming each row represents one ball bowled, so convert to overs
        )
    ''')

# Commit the transaction and close the connection
conn.commit()
conn.close()

print("Database created and populated successfully.")


In [ ]:

import pandas as pd
import mysql.connector

# Load match data
file_path = r'D:\dbms_project\match_data.csv'
match_data = pd.read_csv(file_path)

# Connect to MySQL database
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="Murari@24",
    database="project"
)
cursor = conn.cursor()

# Create Officials table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS Officials (
    official_id INT PRIMARY KEY AUTO_INCREMENT,
    name VARCHAR(100) NOT NULL,
    role VARCHAR(50)
)
''')

# Create MatchOfficials table if it doesn't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS MatchOfficials (
    match_id INT,
    official_id INT,
    role VARCHAR(50),
    FOREIGN KEY (match_id) REFERENCES Matches(match_id),
    FOREIGN KEY (official_id) REFERENCES Officials(official_id),
    PRIMARY KEY (match_id, official_id, role)
)
''')

# Insert data into Officials table and create a dictionary to map names to IDs
officials_map = {}
for role in ['umpire1', 'umpire2', 'tv_umpire', 'match_referee', 'reserve_umpire']:
    officials = match_data[role].dropna().unique()
    for name in officials:
        if name not in officials_map:
            cursor.execute("INSERT IGNORE INTO Officials (name, role) VALUES (%s, %s)", (name, role))
            conn.commit()
            officials_map[name] = cursor.lastrowid

# Insert data into Matches and MatchOfficials tables
for _, row in match_data.iterrows():
    # Prepare values, converting missing data to `None` for SQL NULLs
    match_id = row['match_id']
    date = row['date']
    venue = row['venue']
    city = row['city']
    event = row['event'] if pd.notna(row['event']) else None
    toss_winner = row['toss_winner'] if pd.notna(row['toss_winner']) else None
    toss_decision = row['toss_decision'] if pd.notna(row['toss_decision']) else None
    player_of_match = row['player_of_match'] if pd.notna(row['player_of_match']) else None
    outcome = row['outcome'] if pd.notna(row['outcome']) else None
    winner = row['winner'] if pd.notna(row['winner']) else None
    winner_wickets = row['winner_wickets'] if pd.notna(row['winner_wickets']) else None
    winner_runs = row['winner_runs'] if pd.notna(row['winner_runs']) else None

    # Insert into Matches table
    cursor.execute('''
        INSERT IGNORE INTO Matches (match_id, date, venue, city, event, toss_winner, toss_decision, player_of_match,
                                    outcome, winner, winner_wickets, winner_runs)
        VALUES (%s, %s, %s, %s, %s, 
                (SELECT team_id FROM Teams WHERE team_name = %s), 
                %s, 
                (SELECT player_id FROM Players WHERE name = %s), 
                %s, 
                (SELECT team_id FROM Teams WHERE team_name = %s), 
                %s, %s)
    ''', (match_id, date, venue, city, event, toss_winner, toss_decision, player_of_match, outcome, winner, winner_wickets, winner_runs))

    # Insert officials for each match
    for role in ['umpire1', 'umpire2', 'tv_umpire', 'match_referee', 'reserve_umpire']:
        if pd.notna(row[role]):
            official_id = officials_map.get(row[role])
            cursor.execute('''
                INSERT IGNORE INTO MatchOfficials (match_id, official_id, role)
                VALUES (%s, %s, %s)
            ''', (match_id, official_id, role))

# Commit and close
conn.commit()
conn.close()

print("Match data integrated successfully.")
